In [1]:
import pandas as pd
import os

import helper

In [2]:
agent_df = pd.read_csv(os.path.join('india_base','agent_core_attributes.csv'))

# --- read lookups ---
state_id_lookup = pd.read_csv(os.path.join('reference_data', 'india_census','state_id_lookup.csv'))
state_id_lookup = dict(zip(state_id_lookup['state_name'], state_id_lookup['state_id']))

In [16]:
state_id_lookup

{'jammu_kashmir': 1,
 'himachal_pradesh': 2,
 'punjab': 3,
 'chandigarh': 4,
 'uttarakhand': 5,
 'haryana': 6,
 'nct_of_delhi': 7,
 'rajasthan': 8,
 'uttar_pradesh': 9,
 'bihar': 10,
 'sikkim': 11,
 'arunachal_pradesh': 12,
 'nagaland': 13,
 'manipur': 14,
 'mizoram': 15,
 'tripura': 16,
 'meghalaya': 17,
 'assam': 18,
 'west_bengal': 19,
 'jharkhand': 20,
 'odisha': 21,
 'chhattisgarh': 22,
 'madhya_pradesh': 23,
 'gujarat': 24,
 'daman_diu': 25,
 'dadra_nagar_haveli': 26,
 'maharashtra': 27,
 'andhra_pradesh': 28,
 'karnataka': 29,
 'goa': 30,
 'lakshadweep': 31,
 'kerala': 32,
 'tamil_nadu': 33,
 'puducherry': 34,
 'andaman_nicobar_islands': 35}

In [3]:
# --- Read in 2019 adoption by sector and state, excluding some states ---
res = pd.read_excel(os.path.join('reference_data','Residential and Commercial Installed Capacity by State.xlsx'), sheet_name="Residential")
com = pd.read_excel(os.path.join('reference_data','Residential and Commercial Installed Capacity by State.xlsx'), sheet_name="Commercial")
ind = pd.read_excel(os.path.join('reference_data','Residential and Commercial Installed Capacity by State.xlsx'), sheet_name="Industrial")
res['sector_abbr'] = 'res'
com['sector_abbr'] = 'com'
ind['sector_abbr'] = 'ind'
by_sector = pd.concat([res, com, ind], axis='rows') 
# by_sector[[2018, 2019]] = by_sector[[2017, 2018, 2019]].diff(axis=1)[[2018, 2019]] # convert to annual installations
by_sector = by_sector.fillna(0)
by_sector.columns = ['state_name', 2017, 2018, 2019, 'sector_abbr']
by_sector = by_sector.melt(id_vars=['state_name','sector_abbr'], var_name=['year'], value_name='cum_mw')
# by_sector = by_sector.melt(id_vars=['state_name','sector_abbr'], var_name=['year'], value_name='annual_installed_mw')

replace_dict = {'Delhi':'nct_of_delhi', 'Telangana':'andhra_pradesh'}
by_sector['state_name'] = by_sector['state_name'].replace(replace_dict)

# --- fuzzy string matching ---
clean_list = list(agent_df['state_name'].unique())
by_sector['state_name'] = by_sector['state_name'].apply(helper.sanitize_string)
by_sector['state_name'] = helper.fuzzy_address_matcher(by_sector['state_name'], clean_list)
by_sector['state_id'] = by_sector['state_name'].map(state_id_lookup)
by_sector['state_name'].unique()

# --- group by duplicates ---
by_sector = by_sector.groupby(['state_name', 'state_id', 'sector_abbr','year'], as_index=False)['cum_mw'].sum()

by_sector


,state_name,state_id,sector_abbr,year,cum_mw
0,andaman_nicobar_islands,35,com,2017,0.0
1,andaman_nicobar_islands,35,com,2018,0.0
2,andaman_nicobar_islands,35,com,2019,0.0
3,andaman_nicobar_islands,35,ind,2017,0.0
4,andaman_nicobar_islands,35,ind,2018,0.0
...,...,...,...,...,...
310,west_bengal,19,ind,2018,30.0
311,west_bengal,19,ind,2019,48.0
312,west_bengal,19,res,2017,7.0
313,west_bengal,19,res,2018,24.0


In [4]:
all_india = pd.read_csv(os.path.join('reference_data','ieefa_india_national_mw.csv'))
# all_india['annual_installed_mw'] = all_india['ieefa_total_mw'].diff(1)
all_india

,year,ieefa_total_mw
0,2011,0
1,2012,38
2,2013,117
3,2014,286
4,2015,623
5,2016,1179
6,2017,2177
7,2018,3855
8,2019,5855
9,2020,8855


In [5]:
# --- Scale state/sector to meet annual estimates ---
g = by_sector.groupby(['year'])['cum_mw'].sum()
for y in set(by_sector['year']):
    old_mw = g[y]
    new_mw = all_india.loc[all_india['year']==y, 'ieefa_total_mw'].values[0]
    multiplier = 1 + ((new_mw - old_mw) / old_mw)
    print(y, old_mw, new_mw, multiplier)
    by_sector.loc[by_sector['year'] == y, 'cum_mw'] *= multiplier

# --- Extrapolate state level data to previous years with national sums ---
years = [i for i in set(all_india['year']) if i < by_sector['year'].min()]
years.sort()
years.reverse()
for y in years:
    next_year = by_sector.loc[by_sector['year'] == y+1]
    old_mw = next_year['cum_mw'].sum()
    new_mw = all_india.loc[all_india['year']==y, 'ieefa_total_mw'].values[0]
    multiplier = 1 + ((new_mw - old_mw) / old_mw)
    next_year['cum_mw'] *= multiplier
    next_year['year'] = y
    by_sector = pd.concat([by_sector, next_year], axis='rows')
    print(y, old_mw, new_mw, multiplier)

# --- Extrapolate projected capacity growth forward ---
years = [i for i in set(all_india['year']) if i > by_sector['year'].max()]
years.sort()
for y in years:
    last_year = by_sector.loc[by_sector['year'] == y-1]
    old_mw = last_year['cum_mw'].sum()
    new_mw = all_india.loc[all_india['year']==y, 'ieefa_total_mw'].values[0]
    multiplier = 1 + ((new_mw - old_mw) / old_mw)
    last_year['cum_mw'] *= multiplier
    last_year['year'] = y
    by_sector = pd.concat([by_sector, last_year], axis='rows')
    print(y, old_mw, new_mw, multiplier)


2017 1442.0 2177 1.5097087378640777
2018 2814.0 3855 1.3699360341151385
2019 4429.0 5855 1.3219688417249944
2016 2177.0 1179 0.5415709692237023
2015 1179.0 623 0.5284139100932994
2014 623.0 286 0.4590690208667737
2013 286.0 117 0.40909090909090906
2012 116.99999999999997 38 0.32478632478632485
2011 38.00000000000001 0 0.0
2020 5855.0 8855 1.5123825789923142
2021 8855.0 12855 1.4517221908526257


In [14]:
# --- identify states with no capacity ---
for state in agent_df['state_name'].unique():
    for sector in agent_df['sector_abbr'].unique():
        if sector != 'agg':
            state_mask = (by_sector['state_name'] == state)
            sector_mask = (by_sector['sector_abbr'] == sector)
            sub_df = by_sector.loc[state_mask & sector_mask]
            assert len(sub_df) != 0

# --- Calculate number of adopters from cumulative capacity ---
system_size_dict = {'res':4, 'com':32, 'ind':400} #based on avg rootop size
by_sector['cum_installed_count'] = by_sector['cum_mw'] / by_sector['sector_abbr'].map(system_size_dict) * 1000

# --- create annual installtions ---
by_sector.sort_values(['state_name','sector_abbr','year'], inplace=True)
by_sector['annual_installed_mw'] = by_sector.groupby(['state_name','state_id','sector_abbr'])['cum_mw'].transform(lambda x: x.diff(1))
by_sector['annual_installed_count'] = by_sector.groupby(['state_name','state_id','sector_abbr'])['cum_installed_count'].transform(lambda x: x.diff(1))
by_sector[['cum_mw','cum_mw','annual_installed_mw','annual_installed_count']] = by_sector[['cum_mw','cum_mw','annual_installed_mw','annual_installed_count']].fillna(0).clip(0)

# --- Output annual installations for bass fitting ---
by_sector.to_csv(os.path.join('reference_data', 'clean_pv_installed_all_years.csv'), index=False)

# --- Make cumulative for pv_state_starting_capacities ---
out = by_sector.loc[by_sector['year'] == 2020]
out = out[['state_id','sector_abbr','cum_installed_count','cum_mw']]
out.rename({'cum_installed_count':'pv_systems_count', 'cum_installed_mw':'pv_capacity_mw'}, axis='columns', inplace=True)
out.to_csv(os.path.join('india_base', 'pv_state_starting_capacities.csv'), index=False)

In [9]:
by_sector

,state_name,state_id,sector_abbr,year,cum_mw,cum_installed_count,annual_installed_mw,annual_installed_count
0,andaman_nicobar_islands,35,com,2011,0.000000,0.000000,0.000000,0.000000
0,andaman_nicobar_islands,35,com,2012,0.000000,0.000000,0.000000,0.000000
0,andaman_nicobar_islands,35,com,2013,0.000000,0.000000,0.000000,0.000000
0,andaman_nicobar_islands,35,com,2014,0.000000,0.000000,0.000000,0.000000
0,andaman_nicobar_islands,35,com,2015,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
312,west_bengal,19,res,2017,10.567961,2641.990291,4.844660,1211.165049
313,west_bengal,19,res,2018,32.878465,8219.616205,22.310504,5577.625913
314,west_bengal,19,res,2019,52.878754,13219.688417,20.000289,5000.072213
314,west_bengal,19,res,2020,79.972906,19993.226462,27.094152,6773.538045
